In [1]:
import pandas as pd

import numpy as np
from datetime import datetime

import requests

In [3]:
data = pd.read_csv("./Data/featuresfloatv2.csv")
data=data.loc[:,['verified','statuses','location','date_joined','most_recent_post','following','followers','likes','lists','tweet_language','tweets_this_week','retweet_ratio','retweeted_count','URL works','userNameScore','avg_tweets_by_hour_of_day','avg_tweets_by_day_of_week','account_type']]



In [4]:
data.rename(columns = {'retweet_ratio':'retweet'}, inplace = True)
data['retweet'] = data['retweet'].multiply(2)
data.to_csv('./Data/featuresfloatv3.csv', index=False)

In [6]:
data["URL works"].value_counts(dropna=True)

 0    20454
 1    13588
-1        2
Name: URL works, dtype: int64

crée une relation entre les annes de création (in years) et nombre de following

In [7]:
from datetime import datetime
currentYear = datetime.now().year

data['years_of_join']=0
data.loc[currentYear - data.date_joined==1,'years_of_join']=-1
data.loc[(currentYear - data.date_joined>1) & (currentYear - data.date_joined<=5),'years_of_join']=0
data.loc[currentYear - data.date_joined>5,'years_of_join']=1
data["years_of_join"].value_counts(dropna=True)


 1    32829
 0     1004
-1      211
Name: years_of_join, dtype: int64

If a number is below the average, it means that it is lower than the typical or expected value of the data. 

Conversely, if a number is above the average, it means that it is higher than the typical or expected value of the data.

the avarage of following that a humain account can have 

In [9]:
following_years_avg = data[data['account_type'] == 'human'].groupby('years_of_join')['following'].mean()
following_years_avg = following_years_avg.rename('following_avg')
following_years_avg


years_of_join
-1      71.047904
 0     435.536732
 1    4128.516869
Name: following_avg, dtype: float64

In [166]:

data=data.merge(following_years_avg, on='years_of_join')

data.loc[data.following<=data.following_avg,'following/years']=1
# data.loc[data.followers==data.followers_avg,'followers/years']=0
data.loc[data.following>data.following_avg,'following/years']=-1

In [167]:
print(data["following/years"].value_counts(dropna=True))
data["account_type"].value_counts(dropna=True)

 1.0    31688
-1.0     2356
Name: following/years, dtype: int64


human    22501
bot      11543
Name: account_type, dtype: int64

In [24]:
print(data)

       verified  statuses  location  date_joined  most_recent_post  following  \
0             0      7098         1         2009                 0      166.0   
1             0       270         1         2010              2022      131.0   
2             0     14302         0         2012              2023      440.0   
3             0      4585         1         2009              2023      288.0   
4             1     15851         1         2010              2022       39.0   
...         ...       ...       ...          ...               ...        ...   
34039         1     60100         1         2010              2023      181.0   
34040         1     40277         0         2010              2023      129.0   
34041         0      6679         0         2009              2022      231.0   
34042         1      2580         1         2013              2023       49.0   
34043         0      3566         0         2012              2023    21724.0   

       followers  likes  li

In [156]:
data =data.drop(['years_of_join','followers_avg'], axis=1)

relation entre followers et years_of_last_post

In [14]:
currentYear = datetime.now().year

data['years_of_last_post']=0
# 2023
data.loc[currentYear - data.most_recent_post==0,'years_of_last_post']=1
# 2022 2021
data.loc[(currentYear - data.most_recent_post>0) & (currentYear - data.date_joined<=2),'years_of_last_post']=0
# 2020 ....
data.loc[currentYear - data.most_recent_post>2,'years_of_last_post']=-1
data["years_of_last_post"].value_counts(dropna=True)

 1    15861
-1    13355
 0     4828
Name: years_of_last_post, dtype: int64

In [15]:
followers_years_avg = data[data['account_type'] == 'human'].groupby('years_of_last_post')['followers'].mean()
followers_years_avg = followers_years_avg.rename('followers_avg')
followers_years_avg

years_of_last_post
-1     82474.014106
 0    489794.521803
 1    787054.283611
Name: followers_avg, dtype: float64

In [16]:
data=data.merge(followers_years_avg, on='years_of_last_post')

data.loc[data.followers<=data.followers_avg,'followers/lastpost']=1
# data.loc[data.followers==data.followers_avg,'followers/years']=0
data.loc[data.followers>data.followers_avg,'followers/lastpost']=-1

In [17]:
print(data["followers/lastpost"].value_counts(dropna=True))
data["account_type"].value_counts(dropna=True)

 1.0    30257
-1.0     3787
Name: followers/lastpost, dtype: int64


human    22501
bot      11543
Name: account_type, dtype: int64